In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report

In [11]:
df = pd.read_csv('../data/waterQuality1.csv')
# https://www.kaggle.com/datasets/mssmartypants/water-quality/data

In [12]:
df = df.drop(df[df['ammonia'] == "#NUM!"].index)
df['ammonia'] = df['ammonia'].astype('float')
df['is_safe'] = df['is_safe'].astype('float')

In [13]:
# df.info()

In [14]:

x = np.array(df.drop(['is_safe'], axis=1))
y = np.array(df['is_safe'].copy())

from model_selection import positive_negative_check
positive_negative_check(y)

y = np.where(y <= 0, -1, 1)
positive_negative_check(y)

Positive Class [1.0]: 912 sample.
Negative Class [0.0]: 7084 sample.
Total 7996 Samples.
Positive Class [1]: 912 sample.
Negative Class [-1]: 7084 sample.
Total 7996 Samples.


In [15]:
# from model_selection import train_test_split
from sklearn.model_selection import train_test_split
# buff_x, x_test, buff_y, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
# x_train, x_validate, y_train, y_validate = train_test_split(buff_x,buff_y, test_size=0.2, random_state=42)

buff_x, x_test, buff_y, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
x_train, x_validate, y_train, y_validate = train_test_split(buff_x,buff_y, test_size=0.3, random_state=11)
# x_train

In [16]:
model = svm.SVC(kernel='linear' ,C=10)
model.fit(x_train, y_train)


y_pred = model.predict(x_test)
for i in y_pred:
    print(i)
model.score(x_validate, y_validate)
model.score(x_test, y_test)
    


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred, average="weighted"))
# print("Recall:",metrics.f1_score(y_test, y_pred, average="macro"))
# print(classification_report(y_test, y_pred))


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

In [18]:
from model import SVM
model2 = SVM(kernel='linear', learning_rate=0.001, c=100, max_itr=1900, debug=True, lambda_param=0.01)
model2.fit(x_train, y_train)

predict = model2.predict(x_test)
predict_validate = model2.predict(x_validate)

print(model2.score(predict_validate, y_validate))
print(model2.score(predict, y_test))

C: 100
Learning Rate: 0.001
Lambda Param: 0.01
C: 100
N_Iters: 1900
0.9035956227201668
0.878125
